# FUNÇÃO MAIN PARA RODAR O CODIGO EM LOTE

In [ ]:
#montando o caminho para leitura dos arquivos (certificados, imagens, planilhas, etc)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output
!pip install -q transformers
!pip install -q easyocr
!pip install pymupdf
!pip install pdf2image
!apt-get install poppler-utils
clear_output()

In [ ]:
#Bibliotecas
from datetime import datetime
import pytz
from PIL import Image
from PIL import ImageDraw
from huggingface_hub import hf_hub_download
import fitz
import os
import pandas as pd
import copy
#funcao para converter PDF para png
from pdf2image import convert_from_path
from torchvision import transforms
import torch
#Modelo de deteccao de tabela
from transformers import AutoModelForObjectDetection
#modelo de estrutura
from transformers import TableTransformerForObjectDetection
import numpy as np
import csv
#import easyocr
from tqdm.auto import tqdm
import cv2
from bs4 import BeautifulSoup as bs

Variáveis

In [ ]:
#caminho dos certificados para analise
CERT_PATH = "/content/drive/MyDrive/DataSets/Certificados/In/"

#caminho de saida para geração das anotações das tabelas
OUT_PATH = "/content/drive/MyDrive/DataSets/Certificados/Out/Test/"

#diretorio dos arquivos do GT - Ground Truth (REFERENCIA para comparacao das tabelas)
GT_PATH = "/content/drive/MyDrive/DataSets/Certificados/Out/GT/"

#diretorio de escrita de arquivos
DIROUT_PUBTABLES = "/content/drive/MyDrive/DataSets/Certificados/Out/PubTables/"

Funções de Préprocessamento

In [ ]:
#funcao para varrer os arquivos PDF dos laboratórios nas pastas e retorna um dataframe
#! o método está bem lento (deve verificar o motivo posteriormente)
def InfoPDF(pathPDF):

  lenPages = 0
  doc = fitz.open(pathPDF)

  lenPages = len(doc)
  for pagina in doc:
    isText = bool(pagina.get_text())
    break

  typeArq = ["TEXT" if isText else "IMAGE"]

  return typeArq, lenPages

def listFiles_OLD(CERT_PATH):

  dfArq = pd.DataFrame()
  dirs = [nome for nome in os.listdir(CERT_PATH) if os.path.isdir(os.path.join(CERT_PATH, nome))]

  i = 0
  for dir in dirs:

    #coletando dados do diretorio (id, laboratorio)
    lstDir = dir.split("_")

    if(len(lstDir)==3):

      for file in os.listdir(os.path.join(CERT_PATH, dir)):

        #é arquivo PDF
        if file.lower().endswith(".pdf"):

          pathFile = CERT_PATH + dir + "/" + file
          dfArq.at[i,"LAB"] = lstDir[2]
          dfArq.at[i,'PATH'] = pathFile

          typeArq, qtdPages = InfoPDF(pathFile)
          dfArq.at[i,'TYPE'] = typeArq
          dfArq.at[i,'QTDPAGES'] = qtdPages

          i = i + 1

  return dfArq

def listFiles(CERT_PATH, LAB_PATH):

  dfArq = pd.DataFrame()

  if LAB_PATH is None:
    dirs = [nome for nome in os.listdir(CERT_PATH) if os.path.isdir(os.path.join(CERT_PATH, nome))]
  else:
    dirs = [LAB_PATH]

  i = 0
  for dir in dirs:

    #coletando dados do diretorio (id, laboratorio)
    lstDir = dir.split("_")
    print("lstDir ", lstDir);

    if(len(lstDir)==3):

      for file in os.listdir(os.path.join(CERT_PATH, dir)):

        #é arquivo PDF
        if file.lower().endswith(".pdf"):

          pathFile = CERT_PATH + dir + "/" + file
          print("pathFile ", pathFile);
          dfArq.at[i,"LAB"] = lstDir[2]
          dfArq.at[i,'PATH'] = pathFile

          typeArq, qtdPages = InfoPDF(pathFile)
          dfArq.at[i,'TYPE'] = typeArq
          dfArq.at[i,'QTDPAGES'] = qtdPages

          i = i + 1

  return dfArq

#dfArq = listFiles(CERT_PATH)

def deleteFiles2(dirpath):
  # Obtém a lista de arquivos no diretório
  files = os.listdir(dirpath)

  # Itera sobre os arquivos e os remove
  for file in files:
    filepath = os.path.join(dirpath, file)
    if os.path.isfile(filepath):
      os.remove(filepath)

def is_image_by_extension(file_path):
  image_extensions = ['png', 'jpg', 'jpeg', 'gif', 'bmp', 'tiff', 'webp']  # Adicione outras extensões se necessário
  file_extension = file_path.lower().split('.')[-1]
  return file_extension in image_extensions

def is_image(file_path):
    try:
        # Tenta abrir o arquivo como uma imagem
        Image(file_path)
        return True
    except IOError:
        # Se não for possível abrir como uma imagem, retorna False
        return False

def pdf_page_to_png(pdf_path, page_number, output_path):
  # Convertendo a página do PDF para uma lista de imagens
  images = convert_from_path(pdf_path, first_page=page_number, last_page=page_number)

  # Salvando a imagem como PNG
  images[0].save(output_path, 'PNG')

class MaxResize(object):
  def __init__(self, max_size=800):
      self.max_size = max_size

  def __call__(self, image):
      width, height = image.size
      current_max_size = max(width, height)
      scale = self.max_size / current_max_size
      resized_image = image.resize((int(round(scale*width)), int(round(scale*height))))

      return resized_image

def get_cell_coordinates_by_row(table_data):
  # Extract rows and columns
  rows = [entry for entry in table_data if entry['label'] == 'table row']
  columns = [entry for entry in table_data if entry['label'] == 'table column']

  # Sort rows and columns by their Y and X coordinates, respectively
  rows.sort(key=lambda x: x['bbox'][1])
  columns.sort(key=lambda x: x['bbox'][0])

  # Function to find cell coordinates
  def find_cell_coordinates(row, column):
      cell_bbox = [column['bbox'][0], row['bbox'][1], column['bbox'][2], row['bbox'][3]]
      return cell_bbox

  # Generate cell coordinates and count cells in each row
  cell_coordinates = []

  for row in rows:
      row_cells = []
      for column in columns:
          cell_bbox = find_cell_coordinates(row, column)
          row_cells.append({'column': column['bbox'], 'cell': cell_bbox})

      # Sort cells in the row by X coordinate
      row_cells.sort(key=lambda x: x['column'][0])

      # Append row information to cell_coordinates
      cell_coordinates.append({'row': row['bbox'], 'cells': row_cells, 'cell_count': len(row_cells)})

  # Sort rows from top to bottom
  cell_coordinates.sort(key=lambda x: x['row'][1])

  return cell_coordinates

def aumentar_qualidade_e_contraste(imagem_path, fator_contraste, fator_brilho):
  # Carregar a imagem
  imagem = cv2.imread(imagem_path)

  # Converter a imagem para o espaço de cores LAB (Luminância, Azul, Vermelho)
  lab = cv2.cvtColor(imagem, cv2.COLOR_BGR2LAB)

  # Separar os canais L, A, B
  l, a, b = cv2.split(lab)

  # Aplicar o aumento de contraste na imagem L (luminância)
  l = cv2.add(l, fator_brilho)
  l = cv2.multiply(l, fator_contraste)

  # Mesclar novamente os canais LAB
  lab = cv2.merge((l, a, b))

  # Converter a imagem de volta para o espaço de cores BGR
  imagem_contraste = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

  return imagem_contraste

In [ ]:
#funcoes de pré-processamento dos modelos

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h), (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)


def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def outputs_to_objects(outputs, img_size, id2label):
    m = outputs.logits.softmax(-1).max(-1)
    pred_labels = list(m.indices.detach().cpu().numpy())[0]
    pred_scores = list(m.values.detach().cpu().numpy())[0]
    pred_bboxes = outputs['pred_boxes'].detach().cpu()[0]
    pred_bboxes = [elem.tolist() for elem in rescale_bboxes(pred_bboxes, img_size)]

    objects = []
    for label, score, bbox in zip(pred_labels, pred_scores, pred_bboxes):
        class_label = id2label[int(label)]
        if not class_label == 'no object':
            objects.append({'label': class_label, 'score': float(score),
                            'bbox': [float(elem) for elem in bbox]})

    return objects

def objects_to_crops(img, tokens, objects, class_thresholds, padding=10):
    """
    Process the bounding boxes produced by the table detection model into
    cropped table images and cropped tokens.
    """

    table_crops = []
    for obj in objects:
        if obj['score'] < class_thresholds[obj['label']]:
            continue

        cropped_table = {}

        bbox = obj['bbox']
        bbox = [bbox[0]-padding, bbox[1]-padding, bbox[2]+padding, bbox[3]+padding]

        cropped_img = img.crop(bbox)

        table_tokens = [token for token in tokens if iob(token['bbox'], bbox) >= 0.5]
        for token in table_tokens:
            token['bbox'] = [token['bbox'][0]-bbox[0],
                             token['bbox'][1]-bbox[1],
                             token['bbox'][2]-bbox[0],
                             token['bbox'][3]-bbox[1]]

        # If table is predicted to be rotated, rotate cropped image and tokens/words:
        if obj['label'] == 'table rotated':
            cropped_img = cropped_img.rotate(270, expand=True)
            for token in table_tokens:
                bbox = token['bbox']
                bbox = [cropped_img.size[0]-bbox[3]-1,
                        bbox[0],
                        cropped_img.size[0]-bbox[1]-1,
                        bbox[2]]
                token['bbox'] = bbox

        cropped_table['image'] = cropped_img
        cropped_table['tokens'] = table_tokens

        table_crops.append(cropped_table)

    return table_crops

In [ ]:
#FUNÇÕES DE MANIPULACAO DE IMAGENS E ARQUIVOS

#ler dados da celula
def apply_ocr(cell_coordinates):
    # let's OCR row by row
    data = dict()
    max_num_columns = 0
    for idx, row in enumerate(tqdm(cell_coordinates)):
      row_text = []
      for cell in row["cells"]:
        # crop cell out of image
        cell_image = np.array(cropped_table.crop(cell["cell"]))
        # apply OCR
        result = reader.readtext(np.array(cell_image))
        if len(result) > 0:
          # print([x[1] for x in list(result)])
          text = " ".join([x[1] for x in result])
          row_text.append(text)

      if len(row_text) > max_num_columns:
          max_num_columns = len(row_text)

      data[idx] = row_text

    #print("Max number of columns:", max_num_columns)

    # pad rows which don't have max_num_columns elements
    # to make sure all rows have the same number of columns
    for row, row_data in data.copy().items():
        if len(row_data) != max_num_columns:
          row_data = row_data + ["" for _ in range(max_num_columns - len(row_data))]
        data[row] = row_data

    return data

#similaridade de strings conhecido como "Distância de Levenshtein"
def calcPercSimStrings(str1, str2):

  #retirando quebra de linhas da string
  str1 = str1.replace("\n", " ")
  str2 = str2.replace("\n", " ")

  tamanho_str1 = len(str1)
  tamanho_str2 = len(str2)

  matriz = [[0] * (tamanho_str2 + 1) for _ in range(tamanho_str1 + 1)]

  for i in range(tamanho_str1 + 1):
    matriz[i][0] = i

  for j in range(tamanho_str2 + 1):
    matriz[0][j] = j

  for i in range(1, tamanho_str1 + 1):
    for j in range(1, tamanho_str2 + 1):
        if str1[i - 1] == str2[j - 1]:
            custo_substituicao = 0
        else:
            custo_substituicao = 1
        matriz[i][j] = min(matriz[i - 1][j] + 1,       # Deletar
                            matriz[i][j - 1] + 1,       # Inserir
                              matriz[i - 1][j - 1] + custo_substituicao)  # Substituir

  distancia = matriz[tamanho_str1][tamanho_str2]
  maximo_tamanho = max(tamanho_str1, tamanho_str2)

  similaridade = 0
  if maximo_tamanho > 0:
    similaridade = (maximo_tamanho - distancia) / maximo_tamanho
  #print (" similaridade entre {0} e {1}: {2}".format(str1, str2, similaridade * 100))
  return similaridade * 100

#funcao que compara o valor de duas listas e calcula a media do percentual de similaridade entre eles
#(para calcular o valor do bbox das tabelas e células das tabelas)

def calcPercSimValueLists(lista1, lista2):
  if len(lista1) != len(lista2):
      print("calcPercSimValueLists, listas de tamanhos diferentes, lista1=",lista1,"/ lista2 = ",lista2)
      raise ValueError("As listas devem ter o mesmo comprimento.")

  percSim = [ (1 / (1 + (abs(num1 - num2)))) * 100 for num1, num2 in zip(lista1, lista2)]


  return sum(percSim) / len (percSim)

#(para calcular o percentual de similaridade entre dois números
def calcPercSimValueNums(num1, num2):

  percSim = (1 / (1 + (abs(num1 - num2)))) * 100
  #print (" similaridade entre os numeros {0} e {1}: {2}".format(num1, num2, percSim))
  return percSim

#coletar os arquivos de acordo com premissas (prefixo e sufixo)
def getFilesByPrefix(path, prefix, sufix):
  lstFiles = []
  for fileName in os.listdir(path):
      if prefix in fileName and fileName.endswith(sufix):
          lstFiles.append(fileName)
  return lstFiles

#coletar no arquivo do GT dados das tabelas de uma determinada pagina
def getListFilesGTInfo(curfile, page, path):

  prefix = curfile + "|" + str(page)
  sufix = "_INFO.info"
  lstTables = getFilesByPrefix (path, prefix, sufix)

  return lstTables

def getGTInfo(tableID, curfile, page, path):

  prefix = tableID + "|" + curfile + "|" + str(page)
  sufix = "_INFO.info"
  lstTables = getFilesByPrefix (path, prefix, sufix)

  return lstTables

#coletar as informacoes da tabela do arquivo _INFO e retornar para uma lista
def getListTablesInfo(path, listFiles):

  listTablesInfo = []
  for fileName in listFiles:
    with open(path + fileName, 'r') as file:
      conteudo = file.read()
      listTablesInfo.append(eval(conteudo))

  return listTablesInfo

# verificar se a estrutura dos dois dicionários INFO são similares
def checkDimensionINFO(dicTableGT, dicTable):

  msgErro = "0 - SUCESSO"
  isIdentical = True

  #se o tamanho das chaves dos dicionários são diferentes
  if(dicTableGT.keys() != dicTable.keys()):
    print("Dicionários não possuem o mesmo indice")
    msgErro = "1 - Dicionários não possuem o mesmo indice"
    isIdentical = False

  #se não tiver a mesma dimensao já descarta
  if dicTableGT["DIMENSION"] != dicTable["DIMENSION"]:
    msgErro = "2 - Dicionários não possuem a mesma dimensão"
    isIdentical = False

  #se o tamanho das colunas HEAD e FIRST_LINE não batem
  if(len(dicTableGT["HEAD"]) != len(dicTable["HEAD"])):
    print("Dicionários não possuem o mesmo tamanho da chave HEAD")
    msgErro = "3 - Dicionários não possuem o mesmo tamanho da chave HEAD"
    isIdentical = False

  if(len(dicTableGT["FIRST_LINE"]) != len(dicTable["FIRST_LINE"])):
    msgErro = "4 - Dicionários não possuem o mesmo tamanho da chave FIRST_LINE"
    isIdentical = False

  #return msgErro, isSimilar
  return isIdentical

#calcular similaridade da string das células dos dicionários INFO de mesma dimensão
#(para determinar se dois dicionarios INFO sao similares)
def getPercSimTablesINFO(dicTableGT, dicTable , percTolerancia):

  i = 0
  for textTableGT in dicTableGT["HEAD"]:
    textTable = dicTable["HEAD"][i]
    if (calcPercSimStrings(textTableGT, textTable) < percTolerancia):
      print("Tolerancia entre string ", textTableGT, "e ", textTable,  " menor que ", percTolerancia)
      return False
    i+=1

  i = 0
  for textTableGT in dicTableGT["FIRST_LINE"]:
    textTable = dicTable["FIRST_LINE"][i]
    if (calcPercSimStrings(textTableGT, textTable) < percTolerancia):
      print("Tolerancia entre string ", textTableGT, "e ", textTable,  " menor que ", percTolerancia)
      return False
    i+=1

  return True


#verifica se a dimensão dos dicionários INFO possuem tamanhos parecidos (maximo 1 de diferença)
def isAlmostSimilarINFO(dicTableGT, dicTable):

  #devem possuir a mesma quantidade de linhas e quantidade semelhante de colunas (maximo no modulo 1)

  #colunas
  qtdColGT = len(dicTableGT["HEAD"])
  qtdColTab = len(dicTable["HEAD"])

  #linhas
  qtdLinhasGT = dicTableGT["DIMENSION"].split("X")[0]
  qtdLinhas = dicTable["DIMENSION"].split("X")[0]

  if abs(qtdColGT-qtdColTab) <=1 and qtdLinhasGT == qtdLinhas:
    return True

  return False

#verificar maior valor na lista
def maiorValor(lista):

  maiorValor = 0
  for item in lista:
    if item > maiorValor:
        maiorValor = item

  return maiorValor

#verificar possivel similaridade no cabeçalho dos dicionários INFO
def checkAVGSimilaritiesINFO(dicTableGT, dicTable, percTolerancia):

  qtdColGT = len(dicTableGT["HEAD"])
  qtdColTab = len(dicTable["HEAD"])

  arrSim = []
  arrSummary = []
  limit = 4

  #verificar similaridade nas 3 primeiras colunas
  for i in range(qtdColTab):

    for j in range(qtdColGT):
      strTab = dicTable["HEAD"][i]
      strGT = dicTableGT["HEAD"][j]
      arrSim.append(calcPercSimStrings(strTab, strGT))

    arrSummary.append(maiorValor(arrSim))
    arrSim = []

    if(i>= limit):
      break

  #print(arrSummary)
  avgPerc = sum(arrSummary) / len(arrSummary)
  return avgPerc >= percTolerancia


#verificar possivel similaridade no cabeçalho dos dicionários FIRST_LINE
def checkAVGSimilarities2INFO(dicTableGT, dicTable, percTolerancia):

  qtdColGT = len(dicTableGT["FIRST_LINE"])
  qtdColTab = len(dicTable["FIRST_LINE"])

  arrSim = []
  arrSummary = []
  limit = 3

  #verificar similaridade nas 3 primeiras colunas
  for i in range(qtdColTab):

    for j in range(qtdColGT):
      strTab = dicTable["FIRST_LINE"][i]
      strGT = dicTableGT["FIRST_LINE"][j]
      arrSim.append(calcPercSimStrings(strTab, strGT))

    arrSummary.append(maiorValor(arrSim))
    arrSim = []

    if(i>= limit):
      break

  #print(arrSummary)
  avgPerc = sum(arrSummary) / len(arrSummary)
  return avgPerc >= percTolerancia

def getDicTableInfo(labName, curfile, page, qtdlinhas, qtdcolunas, bbox, head, firstLine):
  dicTableInfo = {}

  dicTableInfo["LAB"] = labName
  dicTableInfo["FILE"] = curfile
  dicTableInfo["PAGE"] = page
  dicTableInfo["TABLEID"] = "TBD"
  dicTableInfo["DIMENSION"] = str(qtdlinhas) + "X" + str(qtdcolunas)
  dicTableInfo["BBOX"] = bbox
  dicTableInfo["HEAD"] = head
  dicTableInfo["FIRST_LINE"] = firstLine

  return dicTableInfo

def SaveDicTableInfo (filePath, dicTableInfo):

  strFile = "{"
  lenDic = len(dicTableInfo.items())
  #print(lenDic)
  i = 0
  for chave, valor in dicTableInfo.items():
    vir = "," if i < lenDic-1 else ""
    if type(valor) == str:
      strFile += "'" + str(chave)+ "':'" + str(valor) + "'" + vir + "\n"
    else:
      strFile += "'" + str(chave) + "':" + str(valor) + vir + "\n"
    i = i + 1

  strFile += "}"
  with open(filePath, 'w') as arquivo:
    arquivo.write(strFile)

#numero de ocorrencias de um numero em uma lista
def numTimes(list, num):

  cont = 0
  for valor in list:
    if valor == num:
      cont+=1

  return cont

def isDecimal(valor):
  try:

    if valor == "NAN" or valor == "nan":
      return False
    else:
      float(valor)
      return True
  except ValueError:
      return False
'''
#calcular similaridades em valores das celulas bbox e tokens de duas listas de tabelas
def calStatsTablesValues(lstTable1, lstTable2):

  lstResTokens = []
  lstResBbox = []
  for item1, item2 in zip(lstTable1, lstTable2):
    listaBbox1 = item1["bbox"]
    listaBbox2 = item2["bbox"]
    str1 = "".join(item1["tokens"])
    str2 = "".join(item2["tokens"])

    lstResBbox.append(round(calcPercSimValueLists(listaBbox1, listaBbox2),2))
    strDec1 = str1.replace(",", ".").strip()
    strDec2 = str2.replace(",", ".").strip()
    #se os valores forem numeros converter para float para calcular similiaridade com maior exatidao
    if (isDecimal(strDec1) and isDecimal(strDec2)):
      lstResTokens.append( round( calcPercSimValueNums(float(strDec1), float(strDec2) ),2) )
    #no caso de string
    else:
      lstResTokens.append(round(calcPercSimStrings(str1, str2),2))

  return lstResTokens, lstResBbox

'''
def noteTokensHTML(df):

  lsthead = []
  lsttd = []
  hasheader = False
  hasdata = False
  dict_tokens_html = {}
  lsthead

  #percorre o dataframe para construir a estrutura html de colunas
  for i in range(len(df)):
    primeiraColuna = True
    j = 0
    for column in df.columns:
        #primeira linha sao os cabeçalhos
        if i == 0:
          hasheader = True
          lsthead.append("<td>")
          lsthead.append(df.at[i,column].replace("'","")) #apenas para teste, comentar depois
          lsthead.append("</td>")
        else:
          hasdata = True
          if(primeiraColuna):
            primeiraColuna = False
            #a partir da 3a linha fecha a linha anterior </tr>
            if(j==0 and i > 1):
              lsttd.append("</tr>")
            lsttd.append("<tr>")
          lsttd.append("<td>")
          lsttd.append(df.at[i,column].replace("'","")) #apenas para teste, comentar depois
          lsttd.append("</td>")

        primeiraColuna = False
        j = j + 1

  if(hasheader):
    lsthead.insert(0,"<thead>")
    lsthead.insert(1,"<tr>")
    lsthead.append("</tr>")
    lsthead.append("</thead>")

  if(hasdata):
    lsttd.insert(0,"<tbody>")
    lsttd.append("</tbody>")

  #se a estrutura tiver completa, adiciona no dicionario tokens
  if(hasheader and hasdata):
    lsthead.extend(lsttd)
  else:
    dict_tokens_html = {"tokens": "vazio"}
    lsthead.extend(dict_tokens_html)

  lsthead.insert(0,"<table>")
  lsthead.extend("</table>")
  return lsthead


def noteListTokensBbox(cell_coordinates, lstData, lstTableRef):
  list_tokens_bbox = []

  #dimensao dos dados
  qtdRowData = len(lstData)
  qtdColData = len(lstData[0])
  print("dimensao Data {0} x {1} ".format(qtdRowData, qtdColData))

  #carregando BBOX de cada celula por linha para uma lista
  qtdlinhasBbox = len(cell_coordinates)
  qtdcolunasBbox = len(cell_coordinates[0]["cells"])
  print("dimensao Bbox {0} x {1} ".format(qtdlinhasBbox, qtdcolunasBbox))


  i = 0
  for row in cell_coordinates:
    j = 0
    for bbox in row["cells"]:
      x1 = round(bbox["cell"][0])
      y1 = round(bbox["cell"][1])
      x2 = round(bbox["cell"][2])
      y2 = round(bbox["cell"][3])

      if(lstTableRef is None):
        dict_tokens_bbox = {'tokens': list(lstData[i][j]), 'bbox': [x1, y1, x2, y2]}
      else:
        dict_tokens_bbox = {'tokens': list(lstData[i][j]), 'bbox': [x1 - lstTableRef[0], y1 - lstTableRef[1], x2 - lstTableRef[0], y2 - lstTableRef[1]]}
      list_tokens_bbox.append(dict_tokens_bbox)
      j+=1
    i+=1

  return list_tokens_bbox

def printMetaDados(dicMetaData):

  strout = []
  strout.append("{ \n")

  if "filename" in dicMetaData:
    strout.append("filename: '" + str(dicMetaData["filename"]) + "',\n")
  else:
    strout[0] = "chave filename não existe na estrutura"
    return strout

  if "split" in dicMetaData:
    strout.append("split: '" + str(dicMetaData["split"]) + "',\n")
  else:
    strout[0] = "chave split não existe na estrutura"
    return strout

  if "imgid" in dicMetaData:
    strout.append("'imgid': " + str(dicMetaData["imgid"]) + ",\n")
  else:
    strout[0] = "chave imgid não existe na estrutura"
    return strout

  if "html" in dicMetaData:

    strout.append("--INICIO HTML \n")
    strout.append("'html': \n {")

    if "cells" in dicMetaData["html"] and "structure" in dicMetaData["html"]:

      if isinstance(dicMetaData["html"]["cells"], list ) and isinstance(dicMetaData["html"]["structure"], list ):

        #varrendo o conteudo da lista dicMetaData["html"]["cells"]
        #que contem as duas sublistas tokens e bbox
        strout.append("--INICIO CELLS \n")
        strout.append("'cells': [\n")
        i = 0
        for arrcells in dicMetaData["html"]["cells"]:

           #print da estrutura dos dicionarios tokens e bbox
           tokens =  arrcells["tokens"]
           bbox =  arrcells["bbox"]
           comma = ","
           if i == len(dicMetaData["html"]["cells"]) -1:
            comma = ""
           else:
            comma = ","

           strout.append("      {'tokens': " + str(tokens) + ", 'bbox': " + str(bbox) + "}" + comma + " \n")
           i = i + 1

        strout.append("] --FIM CELLS\n")

        #print da estrutura do dicionario structure
        if(dicMetaData["html"] is not None and dicMetaData["html"]["structure"] is not None):
          #strout.append("      'structure': [" + str("','".join(dicMetaData["html"]["structure"])) + "' \n")
          strout.append("      'structure': ['" + str("','".join([x for x in dicMetaData["html"]["structure"] if x is not None])) + "' \n")

        else:
          strout.append("      'structure': ['None'] \n")


        strout.append("] --FIM STRUCTURE \n")

      else:
        strout[0] = "chave html/cells ou structure não existe na estrutura"
        return strout

    strout.append("} --FIM HTML\n")
  else:
    strout[0] = "chave html não existe na estrutura"
    return strout

  strout.append("} \n")
  return strout

def saveAnnotationFile(dicMetaData, dirout , numPage):

  vecArq = dicMetaData["filename"].split("/")

  nomeArq = ""
  if(len(vecArq)>0):

    tableID = dicMetaData["imgid"].replace("'","")
    filename = vecArq[len(vecArq)-1].split(".")[0]
    labname = vecArq[len(vecArq)-2].split(".")[0]

    nomeArq = tableID + "|" + filename + "|" + str(numPage) + "_METADADOS.mtd"
    print("gravando arquivo ", nomeArq)

    strout = printMetaDados(dicMetaData)

    labDirOut = dirout + "/" + labname + "/"
    if not os.path.exists(labDirOut):
      os.makedirs(labDirOut)

    with open(labDirOut + nomeArq, 'w') as arquivo:
      for linha in strout:
            arquivo.write(linha)

def getPos(lst, key):

  for k, item in enumerate(lst):
    if item == key:
      return k

  return -1

#FUNCAO DE VERIFICA SE EXISTE O BUG DE TRUNCAR O VALOR ∞
def isBUGInfinito(dicTable, dicTableGT):

  char = ""
  posInf = getPos(dicTableGT["FIRST_LINE"], "∞")
  posInfV = getPos(dicTableGT["FIRST_LINE"], "V")

  #possui valor ∞ na tabela? segue análise
  if posInf >-1:
    char = "∞"
    print("Possui valor ∞ na tabela, posInf", posInf)
    #2 - possuem o mesmo valor de dimensao em DIMENSION
    if dicTable["DIMENSION"] == dicTableGT["DIMENSION"]:
      print("Valor DIMENSION iguais")
      #4 primeiras colunas das duas tabelas possuem o mesmo valor?
      print("checkAVGSimilarities2INFO >=60 perc? ",checkAVGSimilarities2INFO(dicTableGT, dicTable, 80))
      if checkAVGSimilarities2INFO(dicTableGT, dicTable, 60):
        print("Quatro primeiras colunas similares")
        #4 - chave HEAD tem o tamanho um a menos que GT
        if len(dicTable["HEAD"]) == len(dicTableGT["HEAD"])-1 and len(dicTable["FIRST_LINE"]) == len(dicTableGT["FIRST_LINE"])-1:
          #dicTable["FIRST_LINE"].insert(posInf, "∞")
          return True, posInf, "∞"

  #possui valor ∞ na tabela? segue análise
  if posInfV >-1:
    char = "V"
    print("Possui valor V na tabela, posInf", posInf)
    #2 - possuem o mesmo valor de dimensao em DIMENSION
    if dicTable["DIMENSION"] == dicTableGT["DIMENSION"]:
      print("Valor DIMENSION iguais")
      #4 primeiras colunas das duas tabelas possuem o mesmo valor?
      print("checkAVGSimilarities2INFO >=60 perc? ",checkAVGSimilarities2INFO(dicTableGT, dicTable, 80))
      if checkAVGSimilarities2INFO(dicTableGT, dicTable, 60):
        print("Quatro primeiras colunas similares")
        #4 - chave HEAD tem o tamanho um a menos que GT
        if len(dicTable["HEAD"]) == len(dicTableGT["HEAD"])-1 and len(dicTable["FIRST_LINE"]) == len(dicTableGT["FIRST_LINE"])-1:
          #dicTable["FIRST_LINE"].insert(posInf, "∞")
          return True, posInfV, "V"

  return False, -1, ""

import glob

def deleteFiles(dir, ext):
  # Obter todos os arquivos com a extensão especificada
  files = glob.glob(os.path.join(dir, f'*.{ext}'))

  # Remover cada arquivo encontrado
  for file in files:
      try:
          os.remove(file)
          print(f"Arquivo {file} removido com sucesso.")
      except OSError as e:
          print(f"Erro ao remover o arquivo {file}: {e}")

def printHTML2(lst, tipo): #com TAB
  # tipo: RAW (cru) ou PRETTY (html com identações)

  strout = ""
  #strres = ''.join(lst)
  strres = ''.join([str(x) for x in lst])
  if tipo == "PRETTY":
    soup = bs(strres, 'html.parser')
    strout = soup.prettify()
    # Substituir espaços por TAB
    strout = strout.replace("  ", "\t")
  else:
    strout = strres

  return strout

def printElementMetaData(dicMetaData, elem):

  strout = []
  strout.append("[")

  if not "filename" in dicMetaData:
    strout[0] = "chave filename não existe na estrutura"
    return strout

  if not "split" in dicMetaData:
    strout[0] = "chave split não existe na estrutura"
    return strout

  if not "imgid" in dicMetaData:
    strout[0] = "chave imgid não existe na estrutura"
    return strout

  if "html" in dicMetaData:
    if "cells" in dicMetaData["html"] and "structure" in dicMetaData["html"]:
      if isinstance(dicMetaData["html"]["cells"], list ) and isinstance(dicMetaData["html"]["structure"], list ):

        i = 0

        if(elem == "BBOX"):
          #strout.append("{")
          for arrcells in dicMetaData["html"]["cells"]:

            tokens =  arrcells["tokens"]
            bbox =  arrcells["bbox"]
            comma = ","
            if i == len(dicMetaData["html"]["cells"]) -1:
              comma = ""
            else:
              comma = ","

            strout.append("{'tokens': " + str(tokens) + ", 'bbox': " + str(bbox) + "}" + comma + " \n")
            i = i + 1

        elif(elem == "HTML_PRETTY"):
          strout.append(printHTML2(dicMetaData["html"]["structure"], "PRETTY"))

        else:
          html = "'"
          html = html + "','".join([element for element in dicMetaData["html"]["structure"] if element]) + "'"
          strout.append(html)

      else:
        strout[0] = "chave html/cells ou structure não existe na estrutura"
        return strout

  else:
    strout[0] = "chave html não existe na estrutura"
    return strout

  strout.append("]")
  return strout

def saveElementMetadata(dicMetaData, elem, dirout, numPage):

  vecArq = dicMetaData["filename"].split("/")
  #print(vecArq)
  nomeArq = ""
  if(len(vecArq)>0):

    tableID = str(dicMetaData["imgid"]).replace("'","")
    filename = vecArq[len(vecArq)-1].split(".")[0]
    labname = vecArq[len(vecArq)-2].split(".")[0]

    nomeArq = tableID + "|" + filename + "|" + str(numPage) + "_" + elem + "." + elem.lower()
    strout = printElementMetaData(dicMetaData, elem)

    labDirOut = dirout + "/" + labname + "/"

    print("Arquivo de anotação ", elem, " gerado = ",  nomeArq)
    with open(labDirOut + nomeArq, 'w') as arquivo:
      for linha in strout:
          arquivo.write(linha)

#retorna o tableID de maior similiaridade entre os dicionarios GT de comparacao
def getMaiorSimilaridade(dic, listasGT):

  lstDicRes = []

  for dicGT in listasGT:

    lstRes = []

    #ajusta caso necessário a dimensao entre as tabelas
    #print("a tratar...", dic["FIRST_LINE"])
    #print(type(dic["FIRST_LINE"]))
    dic["FIRST_LINE"] = ajustColList(dic["FIRST_LINE"], len(dicGT["FIRST_LINE"]))
    lenDic = len(dic["FIRST_LINE"])

    print("getMaiorSimilaridade, FIRST_LINE GT", dicGT["FIRST_LINE"])
    print("getMaiorSimilaridade, FIRST_LINE ANALISE", dic["FIRST_LINE"])
    for i in range(len(dicGT["FIRST_LINE"])):

      lenDicGT = len(dicGT["FIRST_LINE"])

      if lenDic != lenDicGT: #tamanhos diferentes, retornar vazio
        return ""

      str1 = dic["FIRST_LINE"][i]
      str2 = dicGT["FIRST_LINE"][i]
      strDec1 = str1.replace(",", ".").strip()
      strDec2 = str2.replace(",", ".").strip()

      #se os valores forem numeros converter para float para calcular similiaridade com maior exatidao
      if (isDecimal(strDec1) and isDecimal(strDec2)):
        #print(" Similaridade entre dois numeros ", strDec1, " e ", strDec2, " = ", round(calcPercSimValueNums(float(strDec1), float(strDec2)), 2))
        lstRes.append(round(calcPercSimValueNums(float(strDec1), float(strDec2)), 2))
      #no caso de string
      else:
        #print(" Similaridade entre duas strings ", strDec1, " e ", strDec2, " = ", round(calcPercSimStrings(str1, str2),4))
        lstRes.append(round(calcPercSimStrings(str1, str2),2))

    lstDicRes.append({"TABLEID": dicGT["TABLEID"], "RESULT":lstRes})

  #verificando maior media
  tableId = ""
  maiorMedia = 0
  for dicRes in lstDicRes:
    avg = sum(dicRes["RESULT"]) / len(dicRes["RESULT"])
    print("Media ", avg)
    if avg > maiorMedia:
      tableId = dicRes["TABLEID"]
      maiorMedia = avg

  return tableId

def ajustColList(lst1, qtdColsRef):

  # Calcula o número de colunas de cada lista
  #num_cols_lstRef = len(lstRef)
  num_cols_lst1 = len(lst1) if lst1 else 0

  # Se lst1 tiver menos colunas que lstRef, preenche com NaN
  if num_cols_lst1 < qtdColsRef:
    # Calcula o número de colunas a serem adicionadas
    num_cols_adicionais = qtdColsRef - num_cols_lst1
    # Preenche lst2 com NaN nas novas colunas

    #print("num_cols_adicionais ", num_cols_adicionais)
    for i in range(num_cols_adicionais):
      lst1.append("NAN")

  #se lst1 tiver mais coluna que lstRef, remove as colunas adicionais de lst1
  elif num_cols_lst1 > qtdColsRef:
    # Calcula o número de colunas a serem removidas
    num_cols_adicionais = num_cols_lst1 - qtdColsRef
    for i in range(num_cols_adicionais):
      if (len(lst1)>0):
        del(lst1[len(lst1)-1])

  return lst1

#funcao para ajustar uma lista de acordo com a quantidade de linhas e colunas de referencia
# se tiver a mais linhas ou colunas, adiciona, se tiver menos, remove
def ajustList(lst1, qtdRowsRef, qtdColsRef):

  qtdRows = len(lst1)

  #lsteste = eval(strdata)
  lst1_ajust = []

  #1 - ajustando as colunas
  for lstRow in lst1:
    lst1_ajust.append(ajustColList(lstRow, qtdColsRef))

  #1 - ajustando as linhas
  #se precisar adicionar linhas
  if(qtdRowsRef > qtdRows):
    qtdLinhasAdicionais = qtdRowsRef - qtdRows
    for i in range(qtdLinhasAdicionais):
      if len(lst1_ajust) >0 and len(lst1_ajust[0]) >0:
        lst1_ajust.insert(len(lst1_ajust), ["NAN" for _ in range(len(lst1_ajust[0]))])
  #se precisar remover linhas adicionais
  elif(qtdRows > qtdRowsRef):
    qtdLinhasAdicionais = qtdRows - qtdRowsRef
    for i in range(qtdLinhasAdicionais):
      del(lst1_ajust[len(lst1_ajust)-1])

  return lst1_ajust

#funcao para ajustar a estrutura cell_coordinates em relacao a referencia para possibilitar a comparacao e geracao de estatisticas
def ajustCellCord (cellCord, qtdRowsRef, qtdColsRef):

  lstCoord = [999999, 999999, 999999, 999999] #nova lista de coordenadas
  dicNewCol = {'column': lstCoord, 'cell': lstCoord} #uma nova coluna (celula)
  #nova linha da tabela
  newLine =  "{'row': [99999, 99999, 99999, 99999], \
              'cells': [], \
              'cell_count': 0}"
  dicNewLine = eval(newLine)

  #verificando dimensao da estrutura atual
  qtdRows = 0
  qtdCols = 0
  if cellCord is not None and len(cellCord) >0:
    qtdRows = len(cellCord)
    qtdCols = len(cellCord[0]["cells"])
  #qtdRows = len(cellCord)
  #qtdCols = len(cellCord[0]["cells"])

  #adicionando estrutura inicial para cada quantidade de colunas de referencia
  for i in range(qtdColsRef):
    dicNewLine["cells"].insert(i,dicNewCol)

  #adiciona para cada coluna adicional necessária
  if qtdCols < qtdColsRef:
    qtdColAdicionais = qtdColsRef - qtdCols

    print("Adicionando coluna, qtd = ", qtdColAdicionais)
    for i in range(qtdColAdicionais):
      for row in cellCord:
        row['cells'].append(dicNewCol)

  #removendo uma coluna para cada adicional
  elif qtdCols > qtdColsRef:
    qtdColAdicionais = qtdCols - qtdColsRef

    print("Removendo coluna, qtd = ", qtdColAdicionais)
    for i in range(qtdColAdicionais):
      for row in cellCord:
        row['cells'] = row['cells'][:-1]

  #adiciona linha para cada linha adicional necessária
  if qtdRows < qtdRowsRef:
    qtdRowAdicionais = qtdRowsRef - qtdRows

    print("Adicionando linha, qtd = ", qtdRowAdicionais)
    for i in range(qtdRowAdicionais):
      cellCord.append(dicNewLine)

  #removendo linha para cada linha adicional necessária
  elif qtdRows > qtdRowsRef:
    qtdRowAdicionais = qtdRows - qtdRowsRef

    print("Removendo linha, qtd = ", qtdRowAdicionais)
    #removendo linha para cada adicional
    for i in range(qtdRowAdicionais):
      del(cellCord[len(cellCord)-1])

  return cellCord

def temRepeticoes(lista):
    return len(lista) != len(set(lista))

Pré-processamento (preparação dos modelos)

In [ ]:
#modelo 1 de deteccao de tabela - normalizando hiperparametros de entrada
import easyocr

#preparando processadores para modelo
device = "cuda" if torch.cuda.is_available() else "cpu"
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

############ MODELO 1 - DETEÇÃO DE TABELA ------- ##############

#modelo para detectar as tabelas
model = AutoModelForObjectDetection.from_pretrained("microsoft/table-transformer-detection", revision="no_timm")
model.to(device)

detection_transform = transforms.Compose([
    MaxResize(800),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

############ MODELO 2 - ESTRUTURA DE TABELA ------- ##############

structure_model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-structure-recognition-v1.1-all")
structure_model.to(device)

structure_transform = transforms.Compose([
    MaxResize(1000),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
clear_output()

# FUNCAO PRINCIPAL - GERAR ARQUIVOS DE METADADOS, BBOX E HTML

In [ ]:
################################### FUNCAO PRINCIPAL - GERAR ARQUIVOS DE METADADOS #######################

dfArq = listFiles(CERT_PATH, None) # carregar a lista de arquivos no DATAFRAME

reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

# Definir o fuso horário de SP
fuso_horario_brasilia = pytz.timezone('America/Sao_Paulo')

#varrendo cada arquivo
#varrendo cada pagina do arquivo
#lst = [{'PATH':'/content/drive/MyDrive/DataSets/Certificados/In/LAB_02_PRESERTEC/009.pdf', "QTDPAGES":2}]
#dfArq = pd.DataFrame(lst)

pathLab = DIROUT_PUBTABLES + "/" + LAB_PATH + "/"

if(os.path.exists(pathLab)):
  print('Removendo arquivos gerados anteriormente.., caminho:', pathLab)
  deleteFiles2(pathLab) #deletando os arquivos anteriores

for filepath, pages in zip(dfArq["PATH"], dfArq["QTDPAGES"]):

  arrcurfile = filepath.split("/")
  curFile = arrcurfile[len(arrcurfile)-1]
  labName = arrcurfile[len(arrcurfile)-2]

  #inicializando variaveis GT
  listFilesGT = []
  listTablesInfoGT = []

  #coletando dados das tabelas GT para comparacao e gerar o ID da imagem correto
  GT_LAB_OUT = GT_PATH + "/" + labName + "/"

  #varrendo cada pagina do arquivo
  for i in range(int(pages)):

    page = i+1
    #verificando a quantidade de tabelas por pagina

    #verificando se a pasta do laboratorio existe, caso negativo, cria
    labDirOut = DIROUT_PUBTABLES + "/" + labName + "/"
    if not os.path.exists(labDirOut):
      os.makedirs(labDirOut)

    #definindo variaveis para gravacao do arquivo de saida
    noExtension = curFile.replace(".pdf","")
    #arquivo de PDF de leitura
    #path_pdf_in = dfArq["PATH"][0]
    path_pdf_in = filepath
    #caminho para arquivo PNG convertido
    path_png_out =  labDirOut + labName + "|" + noExtension + "|" + str(page) + ".png"
    #caminho para arquivo PNG convertido e com melhoria de contraste e brilho
    path_png_out_plus =  labDirOut + labName + "|" + noExtension + "|" + str(page) + "_plus.png"

    ####1 - converter pdf para imagem  ####
    pdf_page_to_png(path_pdf_in, page, path_png_out)

    ####2 - melhorar a qualidade melhorar a qualidade da imagem#####
    img_plus = aumentar_qualidade_e_contraste(path_png_out, 1, 5.5)
    cv2.imwrite(path_png_out_plus, img_plus)

    ####3 Carregar imagem para o modelo e tratar contraste
    image = Image.open(path_png_out).convert("RGB")

    ####4 - PREPARAR IMAGEM PARA MODELO 1 (DETECÇÃO DE TABELA)
    pixel_values = detection_transform(image).unsqueeze(0)
    pixel_values = pixel_values.to(device)

    model.to(device)
    outputs = None
    with torch.no_grad():
      outputs = model(pixel_values)

    #print("model.config.id2label ", model.config.id2label)
    #print("pixel_values.shape ", pixel_values.shape)
    #print("outputs.logits.shape ", outputs.logits.shape)

    # update id2label to include "no object"
    id2label = model.config.id2label
    id2label[len(model.config.id2label)] = "no object"

    #coletando coordenadas da tabela (bbox)
    objects = outputs_to_objects(outputs, image.size, id2label)

    tokens = []
    detection_class_thresholds = {
        "table": 0.5,
        "table rotated": 0.5,
        "no object": 10
    }
    crop_padding = 10

    #tabelas coletadas do modelo TSR
    tables_crops = objects_to_crops(image, tokens, objects, detection_class_thresholds, padding=0)

    print("Arquivo: [", path_pdf_in , "] / qtd de tabelas para ler da pagina[" + str(page) + "]: ", len(tables_crops))

    # FOR PARA CADA TABLE DA IMAGEM COLETADA
    for j in range(len(tables_crops)):

      #nova pagina, carrega a lista de referencia (GT) da pagina em questao
      if j==0:
        print("Novo page scan, carregando listTablesInfoGT....")
        listFilesGT = getListFilesGTInfo(noExtension, page, GT_LAB_OUT)
        listTablesInfoGT = getListTablesInfo(GT_LAB_OUT,listFilesGT)

      pathTable =  labDirOut + labName + "|" + noExtension + "|" + str(page) + "|tab" + str(j+1)

      cropped_table = tables_crops[j]['image'].convert("RGB")
      cropped_table.save("table.jpg")

      ####5 - PREPARAR IMAGEM PARA MODELO 2 (ESTRUTURA DE TABELA)
      pixel_values2 = structure_transform(cropped_table).unsqueeze(0)
      pixel_values2 = pixel_values2.to(device)

      # forward pass
      outputs2 = None
      with torch.no_grad():
        outputs2 = structure_model(pixel_values2)

      #==== 5 - GRAVANDO AS INFORMAÇÕES PARA O ARQUIVO INFO ======
      # update id2label to include "no object"
      structure_id2label = structure_model.config.id2label
      structure_id2label[len(structure_id2label)] = "no object"

      cells = outputs_to_objects(outputs2, cropped_table.size, structure_id2label)
      cell_coordinates = get_cell_coordinates_by_row(cells)

      qtdlinhas = 0
      qtdcolunas = 0
      if cell_coordinates is not None and len(cell_coordinates) >0:
        qtdlinhas = len(cell_coordinates)
        qtdcolunas = len(cell_coordinates[0]["cells"])

      print("=============>Tabela pagina {0} caminho: {1} ".format(page,pathTable))
      print("Dimensão cell_cordinates {0} x {1} ".format(qtdlinhas, qtdcolunas))

      #CARREGANDO DADOS DA TABELA LIDA DO MODELO
      data = apply_ocr(cell_coordinates)

      #CARREGANDO E CORRIGINDO OS DADOS (CASO NECESSÁRIO)
      lstData = []
      for row, row_data in data.items():
        lstData.append(row_data)

      #print("objects", objects)
      #print("data", data)
      print("lstData", lstData)
      #print("cell_coordinates", cell_coordinates)

      qtdLinhasData = 0
      qtdColunasData = 0
      if(lstData is not None and len(lstData) >0):
        qtdLinhasData = len(lstData)
        qtdColunasData = len(lstData[0])

      print("Dimensão lstData {0} x {1} ".format(qtdLinhasData, qtdColunasData))

      #coletando o head e primeira linha da tabela para geração da tabela INFO
      l = 0
      lstHead = []
      lstFirst = []
      for row, row_data in data.items():
        if l==0:
          lstHead = list(row_data)
        elif l==1:
          lstFirst = list(row_data)
        elif l==2: #(CASO NORMAL)
          if numTimes(lstHead, "") > 2 or temRepeticoes(lstHead):
            print("Possivel ocorrencia de cabeçalho, pulando 1.... ==> ",lstHead)
            lstHead = copy.deepcopy(lstFirst)
            lstFirst = list(row_data)
        #elif l==2: #CASO ESPECIFICO PARA PRECISOTEC
        #  #if numTimes(lstHead, "") > 2 or temRepeticoes(lstHead):
        #  print("Pulando 2 casas (PRECISOTEC).... ==> ",lstHead)
        #  lstHead = lstFirst
        #  lstFirst = list(row_data)
       #   break
        else:
          pass
        l += 1

      #print("dicTable HEAD antes", dicTable["HEAD"])
      #print("dicTable FIRST_LINE antes", dicTable["FIRST_LINE"])

      #gerando o dicionario para comparacao com GT
      #esse bbox é da TABELA em si
      bbox = [round(objects[j]["bbox"][0]), round(objects[j]["bbox"][1]), round(objects[j]["bbox"][2]), round(objects[j]["bbox"][3])]
      dicTable = getDicTableInfo(labName, noExtension, page, qtdlinhas, qtdcolunas, bbox, lstHead, lstFirst)

      #CASO 1 - VERIFICANDO SE TABELAS SAO IDENTICAS  (DE MESMA DIMENSAO)
      #comparando o dicionario dos dados da tabela com a lista da GT para verificar qual ID será gerado
      equalTables = False
      for dicTableGT in listTablesInfoGT:
        if checkDimensionINFO(dicTableGT, dicTable) and getPercSimTablesINFO(dicTableGT, dicTable , 75):
          dicTable["TABLEID"] = dicTableGT["TABLEID"]
          dicTable["DIMENSION"] = dicTableGT["DIMENSION"]
          dicTable["OBS"] = "CASO 1 - TABELAS IDENTICAS"
          print("CASO 1 - TABELAS IDENTICAS, tableID ", dicTableGT["TABLEID"])
          equalTables = True
          #remove da lista de dicionario o valor do tableid encontrado
          listTablesInfoGT[:] = [dic for dic in listTablesInfoGT if dic.get("TABLEID") != dicTable["TABLEID"]]
          break

      #CASO 2 - BUG DO INFINITO (TABELA TRUNCADA DEVIDO A NAO RECONHECER SIMBOLO DO INFINITO
      isBugInfinito = False
      posInf = -1
      if not equalTables:

        for dicTableGT in listTablesInfoGT:
          #print("CASO 2 - Comparacao, dicTableGT: ", dicTableGT)
          #print("CASO 2 - Comparacao, dicTable antes: ", dicTable)

          isBugInf, posInf, char = isBUGInfinito(dicTable, dicTableGT)
          if isBugInf:
             print("CASO 2 - BUG do infinto confirmado [", dicTableGT["TABLEID"], "] posInf ", posInf)
             dicTable["TABLEID"] = dicTableGT["TABLEID"]
             dicTable["DIMENSION"] = dicTableGT["DIMENSION"]
             dicTable["HEAD"].insert(posInf, "NAN")
             dicTable["FIRST_LINE"].insert(posInf, char)
             dicTable["OBS"] = "CASO 2 - TABELAS SIMILARES (BUG INFINITO)"
             #print("CASO 2 - Comparacao, dicTable depois: ", dicTable)
             isBugInfinito = True
             #remove da lista de dicionario o valor do tableid encontrado
             listTablesInfoGT[:] = [dic for dic in listTablesInfoGT if dic.get("TABLEID") != dicTable["TABLEID"]]
             break

      #CASO 3 - Coletar maior similaridade entre as tabelas da mesma página (de mesma dimensao)
      if not equalTables and not isBugInfinito:
        print("CASO 3 - Coletar maior similaridade entre as tabelas da mesma página")
        dicTable["TABLEID"] = getMaiorSimilaridade(dicTable, listTablesInfoGT)
        listFileGT = getGTInfo(dicTable["TABLEID"], noExtension, page, GT_LAB_OUT)
        listTableInfoGT = getListTablesInfo(GT_LAB_OUT,listFileGT)
        if(len(listTableInfoGT) > 0):
          print("entrou getListTablesInfo ", listTableInfoGT[0]["DIMENSION"])
          dicTable["DIMENSION"] = listTableInfoGT[0]["DIMENSION"]
        dicTable["OBS"] = "CASO 3 - TABELA DE MAIOR SIMILARIDADE"
        print("TABLEID de maior similaridade = ", dicTable["TABLEID"])
        #remove da lista de dicionario o valor do tableid encontrado
        listTablesInfoGT[:] = [dic for dic in listTablesInfoGT if dic.get("TABLEID") != dicTable["TABLEID"]]

      print("dicTable HEAD", dicTable["HEAD"])
      print("dicTable FIRST_LINE", dicTable["FIRST_LINE"])

      if dicTable["DIMENSION"] != 0:
        qtdLinhasGT = int(dicTable["DIMENSION"].split("X")[0])
        qtdColunasGT = int(dicTable["DIMENSION"].split("X")[1])
        print("Dimensão dicTableGT {0} x {1} ".format(qtdLinhasGT, qtdColunasGT))

      #gravar o arquivo INFO da tabela para comparacao com GT
      #encontrou tabela identica ou similar
      if dicTable["TABLEID"] != "TBD" and dicTable["TABLEID"] != "":
        filePath = labDirOut + dicTable["TABLEID"] + "|" + noExtension + "|" + str(page) + "_INFO.info"
        print("gerando arquivo INFO de resultado - SUCESSO: ", filePath)
        SaveDicTableInfo(filePath, dicTable)
      else:
        #no caso de nao ter encontrado tabela similar ao GT para comparação, registrar arquivo de erro
        filePath = labDirOut + noExtension + "|" + str(page) + "|" + "_INFO_ERRO.error"
        dicTable["OBS"] = "ERRO - TABLEID NÃO ENCONTRADO"
        print("gerando arquivo INFO de resultado - ERRO: ", filePath)
        SaveDicTableInfo(filePath, dicTable)

      #==== 5 - GRAVANDO AS INFORMAÇÕES DAS TABELAS (METADADOS, BBOX E HTML) ======

      #encontrou o table ID para comparar com o GT
      if dicTable["TABLEID"] != "TBD" and dicTable["TABLEID"] != "":

        #AJUSTANDO ARRAYS (SE NECESSÁRIO) lsData e cell_coordinates
        if qtdLinhasData != qtdLinhasGT or qtdColunasData != qtdColunasGT: #corrigir lsData
          print("Ajustando lstData  - qtdLinhasData =", qtdLinhasData, " x qtdLinhasGT =", qtdLinhasGT)
          print("Ajustando lstData  - qtdColunasData =", qtdColunasData, " x qtdColunasGT =", qtdColunasGT)
          lstData = ajustList(lstData, qtdLinhasGT, qtdColunasGT)

        if qtdlinhas != qtdLinhasGT or qtdcolunas != qtdColunasGT: #corrigir cell_coordinates
          print("Ajustando cell_coordinates  - qtdlinhas =", qtdlinhas, " x qtdLinhasGT =", qtdLinhasGT)
          print("Ajustando cell_coordinates  - qtdcolunas =", qtdcolunas, " x qtdColunasGT =", qtdColunasGT)
          cell_coordinates = ajustCellCord(cell_coordinates, qtdLinhasGT, qtdColunasGT)

        #print("TABLEID válido, iniciando a coleta de dados (METADADOS E BBOX)...., TABLEID [",dicTable["TABLEID"],"]")
        #modelo do dicmetada
        dicMetaData = {
        "filename": 0,
        "split": "train",
        "imgid": "",
        "html": {
          "cells": 0,
          "structure": 0
                }
        }

        #inicializando a lista
        dicMetaData["filename"] = path_pdf_in
        dicMetaData["imgid"] = dicTable["TABLEID"]

        qtdRowBbox = 0
        qtdColBbox = 0
        #dimensão do BBOX
        if cell_coordinates is not None and len(cell_coordinates) > 0:
          qtdRowBbox = len(cell_coordinates)
          qtdColBbox = len(cell_coordinates[0]["cells"])

        #dimensao dos dados
        qtdRowData = 0
        qtdColData = 0
        if lstData is not None and len(lstData) > 0:
          qtdRowData = len(lstData)
          qtdColData = len(lstData[0])

        print("TABLEID diferente de vazio (APOS TRATAMENTO DAS ESTRUTURAS)...")
        print("qtdRowBbox", qtdRowBbox, " x qtdColBbox", qtdColBbox)
        print("qtdRowData", qtdRowBbox, " x qtdColData", qtdColData)

        #se a dimensão do tokens da celula e do BBOX forem identicos, gravar no arquivo
        if (qtdRowBbox == qtdRowData and qtdColBbox == qtdColData):

          lstCells = []
          lstStructure = []

          #carrega e concatena a lista de tokens das celulas e bbox calculando como referencia as coordenadas da tabela principal
          lstCells.extend(noteListTokensBbox(cell_coordinates, lstData, None))
          dfDados = pd.DataFrame(lstData[0:])
          lstStructure.extend(noteTokensHTML(dfDados)) #carrega e concatena a lista de tokens html
          dicMetaData["html"]["cells"] = lstCells
          dicMetaData["html"]["structure"] = lstStructure

          print("Gravando Metadados: ", DIROUT_PUBTABLES)
          saveAnnotationFile(dicMetaData, DIROUT_PUBTABLES , page)
          saveElementMetadata(dicMetaData, "BBOX", DIROUT_PUBTABLES, page)
          saveElementMetadata(dicMetaData, "HTML", DIROUT_PUBTABLES, page)
          saveElementMetadata(dicMetaData, "HTML_PRETTY", DIROUT_PUBTABLES, page)
          deleteFiles(labDirOut, "png")
        else:
          print("Dimensão diferente entre as tabelas")

    #break #para cada pagina

  #break # para cada arquivo

# Obter a data e hora corrente
data_e_hora_corrente = datetime.now(fuso_horario_brasilia)
# Formatar a data e hora corrente para o formato desejado
data_e_hora_formatadas = data_e_hora_corrente.strftime("%d/%m/%Y %H:%M:%S")
#MAXFILES = 1 #apenas para testes, delimitar a quantidade de certificados a ler
print('FIM DO PROCESSAMENTO ', data_e_hora_formatadas)

# TEDS - CALCULO PARA O MODELO

In [ ]:
from IPython.display import clear_output
!pip install distance
!pip install apted
!pip install lxml
!pip install tqdm
clear_output()

In [ ]:
import distance
from apted import APTED, Config
from apted.helpers import Tree
from lxml import etree, html
from collections import deque
#from parallel import parallel_process
from tqdm import tqdm

class TableTree(Tree):
    def __init__(self, tag, colspan=None, rowspan=None, content=None, *children):
        self.tag = tag
        self.colspan = colspan
        self.rowspan = rowspan
        self.content = content
        self.children = list(children)

    def bracket(self):
        """Show tree using brackets notation"""
        if self.tag == 'td':
            result = '"tag": %s, "colspan": %d, "rowspan": %d, "text": %s' % \
                     (self.tag, self.colspan, self.rowspan, self.content)
        else:
            result = '"tag": %s' % self.tag
        for child in self.children:
            result += child.bracket()
        return "{{{}}}".format(result)


class CustomConfig(Config):
    @staticmethod
    def maximum(*sequences):
        """Get maximum possible value
        """
        return max(map(len, sequences))

    def normalized_distance(self, *sequences):
        """Get distance from 0 to 1
        """
        return float(distance.levenshtein(*sequences)) / self.maximum(*sequences)

    def rename(self, node1, node2):
        """Compares attributes of trees"""
        if (node1.tag != node2.tag) or (node1.colspan != node2.colspan) or (node1.rowspan != node2.rowspan):
            return 1.
        if node1.tag == 'td':
            if node1.content or node2.content:
                return self.normalized_distance(node1.content, node2.content)
        return 0.


class TEDS(object):
    ''' Tree Edit Distance basead Similarity
    '''
    def __init__(self, structure_only=False, n_jobs=1, ignore_nodes=None):
        assert isinstance(n_jobs, int) and (n_jobs >= 1), 'n_jobs must be an integer greather than 1'
        self.structure_only = structure_only
        self.n_jobs = n_jobs
        self.ignore_nodes = ignore_nodes
        self.__tokens__ = []

    def tokenize(self, node):
        ''' Tokenizes table cells
        '''
        self.__tokens__.append('<%s>' % node.tag)
        if node.text is not None:
            self.__tokens__ += list(node.text)
        for n in node.getchildren():
            self.tokenize(n)
        if node.tag != 'unk':
            self.__tokens__.append('</%s>' % node.tag)
        if node.tag != 'td' and node.tail is not None:
            self.__tokens__ += list(node.tail)

    def load_html_tree(self, node, parent=None):
        ''' Converts HTML tree to the format required by apted
        '''
        global __tokens__
        if node.tag == 'td':
            if self.structure_only:
                cell = []
            else:
                self.__tokens__ = []
                self.tokenize(node)
                cell = self.__tokens__[1:-1].copy()
            new_node = TableTree(node.tag,
                                 int(node.attrib.get('colspan', '1')),
                                 int(node.attrib.get('rowspan', '1')),
                                 cell, *deque())
        else:
            new_node = TableTree(node.tag, None, None, None, *deque())
        if parent is not None:
            parent.children.append(new_node)
        if node.tag != 'td':
            for n in node.getchildren():
                self.load_html_tree(n, new_node)
        if parent is None:
            return new_node

    def evaluate(self, pred, true):
        ''' Computes TEDS score between the prediction and the ground truth of a
            given sample
        '''
        if (not pred) or (not true):
            return 0.0
        parser = html.HTMLParser(remove_comments=True, encoding='utf-8')
        pred = html.fromstring(pred, parser=parser)
        true = html.fromstring(true, parser=parser)
        if pred.xpath('body/table') and true.xpath('body/table'):
            pred = pred.xpath('body/table')[0]
            true = true.xpath('body/table')[0]
            if self.ignore_nodes:
                etree.strip_tags(pred, *self.ignore_nodes)
                etree.strip_tags(true, *self.ignore_nodes)
            n_nodes_pred = len(pred.xpath(".//*"))
            n_nodes_true = len(true.xpath(".//*"))
            n_nodes = max(n_nodes_pred, n_nodes_true)
            tree_pred = self.load_html_tree(pred)
            tree_true = self.load_html_tree(true)
            distance = APTED(tree_pred, tree_true, CustomConfig()).compute_edit_distance()
            return 1.0 - (float(distance) / n_nodes)
        else:
            return 0.0

    def batch_evaluate(self, pred_json, true_json):
        ''' Computes TEDS score between the prediction and the ground truth of
            a batch of samples
            @params pred_json: {'FILENAME': 'HTML CODE', ...}
            @params true_json: {'FILENAME': {'html': 'HTML CODE'}, ...}
            @output: {'FILENAME': 'TEDS SCORE', ...}
        '''
        samples = true_json.keys()
        if self.n_jobs == 1:
            scores = [self.evaluate(pred_json.get(filename, ''), true_json[filename]['html']) for filename in tqdm(samples)]
        else:
            inputs = [{'pred': pred_json.get(filename, ''), 'true': true_json[filename]['html']} for filename in samples]
            scores = parallel_process(inputs, self.evaluate, use_kwargs=True, n_jobs=self.n_jobs, front_num=1)
        scores = dict(zip(samples, scores))
        return scores

'''
if __name__ == '__main__':
    import json
    import pprint
    with open('sample_pred.json') as fp:
        pred_json = json.load(fp)
    with open('sample_gt.json') as fp:
        true_json = json.load(fp)
    teds = TEDS(n_jobs=4)
    scores = teds.batch_evaluate(pred_json, true_json)
    pp = pprint.PrettyPrinter()
    pp.pprint(scores)
 '''

"\nif __name__ == '__main__':\n    import json\n    import pprint\n    with open('sample_pred.json') as fp:\n        pred_json = json.load(fp)\n    with open('sample_gt.json') as fp:\n        true_json = json.load(fp)\n    teds = TEDS(n_jobs=4)\n    scores = teds.batch_evaluate(pred_json, true_json)\n    pp = pprint.PrettyPrinter()\n    pp.pprint(scores)\n "

In [ ]:
'''
#FUNCAO DE VERIFICA SE EXISTE O BUG DE TRUNCAR O VALOR ∞

def isBUGInfinito(dicTable, dicTableGT):

  posInf = getPos(dicTableGT["FIRST_LINE"], "∞")

  #possui valor ∞ na tabela? segue análise
  if posInf >-1:
    print("Possui valor ∞ na tabela")
    #2 - possuem o mesmo valor de dimensao em DIMENSION
    if dicTable["DIMENSION"] == dicTableGT["DIMENSION"]:
      print("Valor DIMENSION iguais")
      #3 primeiras colunas das duas tabelas possuem o mesmo valor?
      if checkAVGSimilarities2INFO(dicTableGT, dicTable, 80):
        print("Três primeiras colunas similares")
        #4 - chave HEAD tem o tamanho um a menos que GT
        if len(dicTable["HEAD"]) == len(dicTableGT["HEAD"])-1 and len(dicTable["FIRST_LINE"]) == len(dicTableGT["FIRST_LINE"])-1:
          #dicTable["FIRST_LINE"].insert(posInf, "∞")
          return True, posInf

  return False, -1

#funcao para ajustar o valor

isBUGInfinito(dicTable, dicTableGT)
'''

In [ ]:
'''
#verificar possivel similaridade no cabeçalho dos dicionários INFO
def checkINFOFirstLine(dicTableGT, dicTable, percTolerancia):

  qtdColGT = len(dicTableGT["FIRST_LINE"])
  qtdColTab = len(dicTable["FIRST_LINE"])

  arrSim = []
  arrSummary = []
  limit = 4

  #verificar similaridade nas 3 primeiras colunas
  for i in range(qtdColTab):

    for j in range(qtdColGT):
      strTab = dicTable["FIRST_LINE"][i]
      strGT = dicTableGT["FIRST_LINE"][j]
      arrSim.append(calcPercSimStrings(strTab, strGT))

    arrSummary.append(maiorValor(arrSim))
    arrSim = []

  #print(arrSummary)
  avgPerc = sum(arrSummary) / len(arrSummary)
  return avgPerc >= percTolerancia
  '''

# 5 - Função MAIN para gerar estatísticas (ARQUIVOS GT VERSUS IMAGE2TABLE)

Funcoes basicas para estatisticas

In [ ]:
#montando o caminho para leitura dos arquivos (certificados, imagens, planilhas, etc)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import glob
import json

#coletar as informacoes da tabela do arquivo _INFO e retornar para uma lista
def getListTablesInfo(path, listFiles):

  listTablesInfo = []
  for fileName in listFiles:
    with open(path + fileName, 'r') as file:
      conteudo = file.read()
      listTablesInfo.append(eval(conteudo))

  return listTablesInfo

def sortList(lst, len):
    def personList(item):
        # Extrai o número após 'CTM' e converte para inteiro
        return int(item[len:])

    return sorted(lst, key=personList)

#coletar arquivo de acordo com premissas (prefixo e sufixo)
def getFileByPrefix(path, prefix, sufix):

  for fileName in os.listdir(path):
      if prefix in fileName and fileName.endswith(sufix):
          return fileName
  return ""

def getFiles(folderDir, ext):
  # Construir o padrão de busca usando a extensão fornecida
  pattern = os.path.join(folderDir, f"*.{ext}")

  files = []
  arrPath = folderDir.split("/")

  #print("arrPath ", arrPath)
  if len(arrPath) >0 and len(arrPath[len(arrPath)-1].split("_")) >0:
    #print(folderDir)
    lab = arrPath[len(arrPath)-1].split("_")[2]
    # Usar a função glob para encontrar os arquivos correspondentes ao padrão
    files = glob.glob(pattern)

  # Retornar a lista de arquivos encontrados
  return sorted(files)

def getInfoFiles(list):

  lstInfoFiles = []
  for path in list:

    arrPath = path.split("/")
    file = arrPath[len(arrPath)-1]
    tableId = file.split("|")[0]
    fileName = file.split("|")[1]
    page = file.split("|")[2].split("_")[0]

    lstInfoFiles.append({"TABLEID":tableId, "TABLEID":tableId, "FILE":fileName, "PAGE":page})

  return lstInfoFiles

#funcao que compara o valor de duas listas e calcula a media do percentual de similaridade entre eles
#(para calcular o valor do bbox das tabelas e células das tabelas)
def calcPercSimValueLists(lista1, lista2):
  if len(lista1) != len(lista2):
      print("calcPercSimValueLists, listas de tamanhos diferentes, lista1=",lista1,"/ lista2 = ",lista2)
      raise ValueError("As listas devem ter o mesmo comprimento.")

  percSim = [ (1 / (1 + (abs(num1 - num2)))) * 100 for num1, num2 in zip(lista1, lista2)]
  #print("percSim ", percSim)
  #print("result percSim ", sum(percSim) / len (percSim))
  return sum(percSim) / len (percSim)

#(para calcular o percentual de similaridade entre dois números
def calcPercSimValueNums(num1, num2):

  percSim = (1 / (1 + (abs(num1 - num2)))) * 100
  #print (" similaridade entre os numeros {0} e {1}: {2}".format(num1, num2, percSim))
  return percSim

#similaridade de strings conhecido como "Distância de Levenshtein"
def calcPercSimStrings(str1, str2):

  #retirando quebra de linhas da string
  str1 = str1.replace("\n", " ")
  str2 = str2.replace("\n", " ")

  tamanho_str1 = len(str1)
  tamanho_str2 = len(str2)

  matriz = [[0] * (tamanho_str2 + 1) for _ in range(tamanho_str1 + 1)]

  for i in range(tamanho_str1 + 1):
    matriz[i][0] = i

  for j in range(tamanho_str2 + 1):
    matriz[0][j] = j

  for i in range(1, tamanho_str1 + 1):
    for j in range(1, tamanho_str2 + 1):
        if str1[i - 1] == str2[j - 1]:
            custo_substituicao = 0
        else:
            custo_substituicao = 1
        matriz[i][j] = min(matriz[i - 1][j] + 1,       # Deletar
                            matriz[i][j - 1] + 1,       # Inserir
                              matriz[i - 1][j - 1] + custo_substituicao)  # Substituir

  distancia = matriz[tamanho_str1][tamanho_str2]
  maximo_tamanho = max(tamanho_str1, tamanho_str2)

  similaridade = 0
  if maximo_tamanho > 0:
    similaridade = (maximo_tamanho - distancia) / maximo_tamanho
  #print (" similaridade entre {0} e {1}: {2}".format(str1, str2, similaridade * 100))
  return similaridade * 100

#numero de ocorrencias de um numero em uma lista
def numTimes(list, num):

  cont = 0
  for valor in list:
    if valor == num:
      cont+=1

  return cont

#numero de ocorrencias de um numero ser maior ou igual que um numero
def numTimesMoreThen(list, num):

  cont = 0
  for valor in list:
    if valor >= num and valor <100:
      cont+=1

  return cont

def isDecimal(valor):
  try:
      float(valor)
      return True
  except ValueError:
      return False

#calcular similaridades em valores das celulas bbox e tokens de duas listas de tabelas
def calStatsTablesValues(lstTable1, lstTable2):

  lstResTokens = []
  lstResBbox = []
  for item1, item2 in zip(lstTable1, lstTable2):
    listaBbox1 = item1["bbox"]
    listaBbox2 = item2["bbox"]
    str1 = "".join(item1["tokens"])
    str2 = "".join(item2["tokens"])

    if len(listaBbox1) >0 and len(listaBbox2) >0:
      lstResBbox.append(round(calcPercSimValueLists(listaBbox1, listaBbox2),2))
    else:
      lstResBbox.append(0)

    strDec1 = str1.replace(",", ".").strip()
    strDec2 = str2.replace(",", ".").strip()
    #se os valores forem numeros converter para float para calcular similiaridade com maior exatidao
    if (isDecimal(strDec1) and isDecimal(strDec2)):
      lstResTokens.append( round( calcPercSimValueNums(float(strDec1), float(strDec2) ),2) )
    #no caso de string
    else:
      lstResTokens.append(round(calcPercSimStrings(str1, str2),2))

  return lstResTokens, lstResBbox

#calcular quantidade de valores não lidos pelo modelo (NAN ou 999999)
def calStatsNAN(lstTable):

  qtdNANToken = 0
  qtdNANBbox = 0

  for item in lstTable:
    listaBbox = item["bbox"]
    token = "".join(item["tokens"])

    if numTimes(listaBbox, 999999) ==4:
      qtdNANBbox+=1

    if token == "NAN":
      qtdNANToken+=1

  return qtdNANToken, qtdNANBbox

def readFile(filePath):
  try:
    with open(filePath, 'r') as arquivo:
        conteudo = arquivo.read()
    return conteudo
  except FileNotFoundError:
    print(f'O arquivo "{filePath}" não foi encontrado.')
    return None
  except Exception as e:
    print(f'Ocorreu um erro ao ler o arquivo: {e}')
    return None

def SaveFileStats (filePath, dicTableStats):

  strFile = json.dumps(dicTableStats)
  strFile = strFile.replace(",",",\n")

  print("Salvando arquivo de estatísticas: ",filePath)
  with open(filePath, 'w') as arquivo:
    arquivo.write(strFile)

def strInDic(dicionario, string):

  for chave, valor in dicionario.items():
      if isinstance(valor, str) and string in valor:
          return True
  return False

def strInList(lstDic, string):

  for dic in lstDic:
    for chave, valor in dic.items():
        if isinstance(valor, str) and string == valor:
            return True
  return False

def ExportCSVSummary(ToolPath, GTPath):
  #coletando os diretorios dos laboratorios
  dirLabs = [nome for nome in os.listdir(ToolPath)]

  listStats = []
  filesSTATS = []

  lstTableIdGT = []
  #colentando os arquivos de statisticas dos laboratorios
  for dirLab in dirLabs:

    labPath =  ToolPath + dirLab
    filesSTATS = getFiles(labPath, "stats")

    #print("filesSTATS ", filesSTATS)
    #coletando arquivos de estatisticas
    for fileSTATS in filesSTATS:
      dicStats = ast.literal_eval(readFile(fileSTATS))
      listStats.append(dicStats)

    #coletando os tablesID do GT para comparacao
    filesGTInfo = GTPath + dirLab
    filesInfo = getFiles(filesGTInfo, "info")

    #print("filesInfo ", filesInfo)

    for fileInfo in filesInfo:
      arrFile = fileInfo.split("/")
      tableId = arrFile[len(arrFile)-1].split("|")[0]
      lstTableIdGT.append(dirLab+"|"+tableId)

  lstErros = []

  for item in lstTableIdGT:

    #print(item)
    lab = item.split("|")[0]
    tableId = item.split("|")[1]
    if not strInList(listStats, tableId): #não encontrou, adicionar ao erro

      print("Não encontrou TABLEID ", tableId, ", adicionando....")
      #print("GTPath ", GTPath)
      #print("lab ", lab)
      #print("tableId ", tableId)
      print("fileInfo ", fileInfo)

      #coletando informacoes do statsInfo
      #print("parametros a carregar na funcao getFileByPrefix", GTPath + lab, tableId+"|", "info")
      fileInfo = getFileByPrefix(GTPath + lab, tableId+"|", "info")
      #pathInfo = GTPath + dirLab + fileInfo
      lstFile = [fileInfo]
      #print("parametros a carregar na funcao getListTablesInfo", GTPath + lab + "/", lstFile)
      lstInfo = getListTablesInfo(GTPath + lab + "/", lstFile)

      dicTableStats = {}
      dicTableStats["LAB"] = lstInfo[0]["LAB"]
      dicTableStats["FILE"] = lstInfo[0]["FILE"]
      dicTableStats["PAGE"] = lstInfo[0]["PAGE"]
      dicTableStats["TABLEID"] = lstInfo[0]["TABLEID"]
      dicTableStats["DIMENSION"] = lstInfo[0]["DIMENSION"]
      dicTableStats["QTDCELLS"] = 0
      dicTableStats["QTDACERTOSCELLS"] = 0
      dicTableStats["PERCACERTOSCELLS"] = 0
      dicTableStats["PERCACERTOSBBOX"] = 0
      dicTableStats["QTDNAOLIDOSBBOX"] = 0
      dicTableStats["PERCNAOLIDOSBBOX"] = 0
      dicTableStats["QTDNAOLIDOSTOKEN"] = 0
      dicTableStats["PERCNAOLIDOSTOKEN"] = 0
      dicTableStats["TEDS"] = 0

      lstErros.append(dicTableStats)

  lstTotal = listStats + lstErros

  dtStats = pd.DataFrame(lstTotal)
  dtStatsORD = dtStats.sort_values(by=['LAB','FILE', 'PAGE'])
  #print(dtStatsORD)
  print("Arquivo de sumário gerado ", ToolPath + "Summary.xlsx")
  dtStatsORD.to_excel(ToolPath + "Summary.xlsx", index=False)  # index=False para não incluir o índice do DataFrame
  return dtStatsORD

In [ ]:
#gerando estatísticas

import os
import ast
import pandas as pd

#diretorio do img2tableDir
pubTables = "/content/drive/MyDrive/DataSets/Certificados/Out/PubTables/"
#diretorio de referencia GT
GTDir = "/content/drive/MyDrive/DataSets/Certificados/Out/GT/"

#coletando os diretorios dos laboratorios
#dirLabs = [nome for nome in os.listdir(pubTables)]

dirLabs = [] #INFORMAR A LISTA DE PASTAS DE ARQUIVOS POR LABORATÓRIO AQUI

for dirLab in dirLabs:

  if not os.path.isfile(dirLab):
    labPath =  pubTables + dirLab
    labPathGT =  GTDir + dirLab
    #coleta os arquivos info para analise dos tablesID
    filesINFO = getFiles(labPath, "info")
    #coleta os tablesID para comparacao
    lstInfoFiles = getInfoFiles(filesINFO)

    #para cada tableID, gerar estatísticas
    for dicInfoFile in lstInfoFiles:

      print("dicInfoFile", dicInfoFile)
      fileInfo = dicInfoFile["TABLEID"] + "|" + dicInfoFile["FILE"] + "|" + dicInfoFile["PAGE"] + "_INFO.info"
      fileTokenBbox = dicInfoFile["TABLEID"] + "|" + dicInfoFile["FILE"] + "|" + dicInfoFile["PAGE"] + "_BBOX.bbox"
      fileHTML = dicInfoFile["TABLEID"] + "|" + dicInfoFile["FILE"] + "|" + dicInfoFile["PAGE"] + "_HTML.html"

      pathFileInfo = labPath + "/" + fileInfo
      pathFileTokenBbox = labPath + "/" + fileTokenBbox
      pathFileHTML = labPath + "/" + fileHTML

      pathFileInfoGT = labPathGT + "/" + fileInfo
      pathFileTokenBboxGT = labPathGT + "/" + fileTokenBbox
      pathFileHTMLGT = labPathGT + "/" + fileHTML
      print("pathFileInfo =", pathFileInfo)

      #gerando estatisticas do token e bbox
      if os.path.exists(pathFileHTMLGT) and os.path.exists(pathFileTokenBbox) and os.path.exists(pathFileTokenBboxGT):

        #carrega lista de tokens/bbox e HTMLs para comparacao do arquivo corrente com GT
        #arquivo info GT (.info)
        lstInfo = ast.literal_eval(readFile(pathFileInfoGT))
        #arquivo bbox (.bbox)
        lstTkBox = ast.literal_eval(readFile(pathFileTokenBbox))
        #arquivo bbox GT (.bbox)

        lstTkBoxGT = ast.literal_eval(readFile(pathFileTokenBboxGT))
        #arquivo html (.html)
        strHTML = "".join(ast.literal_eval(readFile(pathFileHTML).replace("\n", " ")))
        #arquivo html GT(.html)
        strHTMLGT = "".join(ast.literal_eval(readFile(pathFileHTMLGT).replace("\n", " ")))
        #break

        #TEDS apenas funciona se tiver na estrutura html as tags html e body
        if "<body>" not in strHTML:
          strHTML = "<body>" + strHTML + "</body>"
        if "<html>" not in strHTML:
          strHTML = "<html>" + strHTML + "</html>"
        if "<body>" not in strHTMLGT:
          strHTMLGT = "<body>" + strHTMLGT + "</body>"
        if "<html>" not in strHTMLGT:
          strHTMLGT = "<html>" + strHTMLGT + "</html>"

        qtdLinhas = int(lstInfo["DIMENSION"].split("X")[0])
        qtdColunas = int(lstInfo["DIMENSION"].split("X")[1])
        qtdCells = qtdLinhas * qtdColunas

        #print("lstTkBox",lstTkBox)
        #print("lstTkBoxGT",lstTkBoxGT)
        lstStatsTokens, lstStatsBbox = calStatsTablesValues(lstTkBox, lstTkBoxGT)

        qtdNANToken, qtdNANBbox = calStatsNAN(lstTkBox)

        #calcula qtd de acertos (com similaridade 100% entre os valores dos tokens e tabelas)
        qtdAcertosTokens = numTimes(lstStatsTokens, 100.0)
        #qtdAcertosBbox = numTimes(lstStatsBbox, 100.0)
        percAcertosBbox = round((sum(lstStatsBbox) / len(lstStatsBbox))/100, 2)

        #calcula similidade maior que 95%
        qtdTokensMaior95 = numTimesMoreThen(lstStatsTokens, 95)
        qtdBboxMaior95 = numTimesMoreThen(lstStatsBbox, 95)

        #calcula TEDS entre as estruturas HTMLs (img2table VS GT)
        teds = TEDS()

        scoreTEDS = round(teds.evaluate(strHTML, strHTMLGT), 6)

        #salvado arquivo de estatística
        fileStats = dicInfoFile["TABLEID"] + "|" + dicInfoFile["FILE"] + "|" + dicInfoFile["PAGE"] + "_STATS.stats"
        pathFileStats = labPath + "/" + fileStats

        percAcertosTokens = round((qtdAcertosTokens / qtdCells), 2)
        percNaoLidosBbox = round((qtdNANBbox / qtdCells), 2)
        percNaoLidosToken = round((qtdNANToken / qtdCells), 2)

        dicTableStats = {}
        dicTableStats["LAB"] = lstInfo["LAB"]
        dicTableStats["FILE"] = dicInfoFile["FILE"]
        dicTableStats["PAGE"] = dicInfoFile["PAGE"]
        dicTableStats["TABLEID"] = dicInfoFile["TABLEID"]
        dicTableStats["DIMENSION"] = lstInfo["DIMENSION"]
        dicTableStats["QTDCELLS"] = qtdCells
        dicTableStats["QTDACERTOSCELLS"] = qtdAcertosTokens
        dicTableStats["PERCACERTOSCELLS"] = percAcertosTokens
        dicTableStats["PERCACERTOSBBOX"] = percAcertosBbox
        dicTableStats["QTDNAOLIDOSBBOX"] = qtdNANBbox
        dicTableStats["PERCNAOLIDOSBBOX"] = percNaoLidosBbox
        dicTableStats["QTDNAOLIDOSTOKEN"] = qtdNANToken
        dicTableStats["PERCNAOLIDOSTOKEN"] = percNaoLidosToken
        #dicTableStats["QTDCELLSMAIOR95"] = qtdTokensMaior95
        #dicTableStats["QTDABBOXMAIOR95"] = qtdBboxMaior95
        dicTableStats["TEDS"] = scoreTEDS

        SaveFileStats (pathFileStats, dicTableStats)

        print("Dimensão da tabela ", lstInfo["DIMENSION"], ", total ", str(qtdCells), " células" )
        print("qtdCells:",qtdCells)
        print("qtdAcertosTokens:",qtdAcertosTokens)
        print("percAcertosTokens:",percAcertosTokens,"/",(percAcertosTokens*100),"%")
        print("percAcertosBbox:",percAcertosBbox,"/",(percAcertosBbox*100),"%")
        print("qtdNaoLidosBbox:",qtdNANBbox)
        print("percNaoLidosBbox:",percNaoLidosBbox,"/",(percNaoLidosBbox*100),"%")
        print("qtdNaoLidosTokens:",qtdNANToken)
        print("percNaoLidosToken:",percNaoLidosToken,"/",(percNaoLidosToken*100),"%")
        #print("qtdAcertosTokens>95:",qtdTokensMaior95)
        #print("qtdAcertosBbox>95:",qtdBboxMaior95)
        print('TEDS score:', scoreTEDS,"/",round((scoreTEDS*100),2),"%")
        #break # fim primeiro for

#depois de gerar arquivos de estatística, montar sumário no EXCEL
ExportCSVSummary(pubTables, GTDir)